In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\PracticeProjects\\NL_Text_Summarization\\NL_TEXT_SUMMARIZER'

In [4]:
# This block will later be moved to entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from NLPTextSummarizer.constants import *
from NLPTextSummarizer.utils.common import read_yaml, create_directories

In [6]:
# 4. Update the configuration manager(./src/project_name/config/configuration.py)
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        #print(CONFIG_FILE_PATH)
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath) 
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        # Move config path dict from config.yaml
        config = self.config.data_ingestion
        # create the directories as mentioned in config.yaml
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir 
        )
        return data_ingestion_config



In [9]:
# 5. Update the components (./src/project_name/components/__init__.py)
import os
import urllib.request as request
import zipfile
from NLPTextSummarizer.logging import logger
from NLPTextSummarizer.utils.common import get_size

class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} donloaded with follwoing info:\n {headers}")
        else:
            logger.info(f"{self.config.local_data_file} already exists of size: {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):
        '''
        zip_file_path = str
        Extarcts the zip file into the data directory
        Function return None
        '''
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
# Pipeline component
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-05-05 13:28:35,581: INFO: common: yaml file: D:\PracticeProjects\NL_Text_Summarization\NL_TEXT_SUMMARIZER\config\config.yaml loaded successfully]
[2024-05-05 13:28:35,583: INFO: common: yaml file: D:\PracticeProjects\NL_Text_Summarization\NL_TEXT_SUMMARIZER\params.yaml loaded successfully]
[2024-05-05 13:28:35,584: INFO: common: created directory at: artifacts]
[2024-05-05 13:28:35,585: INFO: common: created directory at: artifacts/data_ingestion]
[2024-05-05 13:28:35,586: INFO: 677450898: artifacts/data_ingestion/data.zip already exists of size: ~ 7718 KB]
